**Problem of small files**
- Reading data can be faster if we get lucky with  `File Skipping` with Delta Log
- the huge number of smaller files makes the process slow because we are going to get multiple unnecessary tasks which will slow down our program
- So we consolidate the large number of files by grouping them into smaller number of groups
- Usually we will be looking for data consolidation.
- open question for later is which data should be putted together, into the same file
- `Optimize` table_name :- it aims to produnce evenly balanced data files with respect to their size on disk.Default is 1GB

In [0]:
%sql
SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite=false;
SET spark.databricks.delta.properties.defaults.autoOptimize.autoCompact=false;

key,value
spark.databricks.delta.properties.defaults.autoOptimize.autoCompact,false


In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/small_files",True)

True

In [0]:
%sql
DROP TABLE IF EXISTS small_files;

CREATE OR REPLACE TABLE small_files (id INT, data STRING)

In [0]:
%python 
for x in range (1,100):
    query=f'''
    INSERT INTO small_files VALUES ( {x},"name{x}" )
    '''

    spark.sql(query)

In [0]:
%sql
DESC DETAIL small_files;
-- 99 FILES CREATED

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,611fc1c3-a6fe-4adf-852b-d6b7ad490440,spark_catalog.default.small_files,null,dbfs:/user/hive/warehouse/small_files,2025-02-03T09:08:35.255Z,2025-02-03T09:11:28Z,List(),List(),99,70907,"Map(delta.autoOptimize.optimizeWrite -> false, delta.autoOptimize.autoCompact -> false)",1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql
SELECT AVG(id) FROM small_files

avg(id)
50.0


Table With Bigger Files

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/big_files",True)

True

In [0]:
%sql
DROP TABLE IF EXISTS big_files;

CREATE OR REPLACE TABLE big_files (id INT, data String);

In [0]:
%python 
values = ", ".join([f"({x},'name{x}')" for x in range(1,100)])
query=f"INSERT INTO big_files VALUES {values}"   
 
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
 DESC DETAIL big_files
 -- SINGLE FILE CREATED

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,47fc0778-0e2e-478d-8f4f-fa69ecada659,spark_catalog.default.big_files,null,dbfs:/user/hive/warehouse/big_files,2025-02-03T09:31:53.432Z,2025-02-03T09:31:56Z,List(),List(),1,1515,"Map(delta.autoOptimize.optimizeWrite -> false, delta.autoOptimize.autoCompact -> false)",1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql
SELECT AVG(id) FROM big_files;
-- this took lesser time then the small_files
-- to understand more we can look in the spark UI

avg(id)
50.0


Optimize table with small files

In [0]:
%sql
DESC HISTORY small_files

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
100,2025-02-03T09:11:55Z,5755764547042441,inaya998877@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(4239566892388298),0203-052121-nggyovx3,99,SnapshotIsolation,false,"Map(numRemovedFiles -> 99, numRemovedBytes -> 70907, p25FileSize -> 1515, numDeletionVectorsRemoved -> 0, minFileSize -> 1515, numAddedFiles -> 1, maxFileSize -> 1515, p75FileSize -> 1515, p50FileSize -> 1515, numAddedBytes -> 1515)",null,Databricks-Runtime/15.4.x-scala2.12
99,2025-02-03T09:11:28Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,98,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
98,2025-02-03T09:11:26Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,97,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
97,2025-02-03T09:11:23Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,96,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
96,2025-02-03T09:11:22Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,95,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
95,2025-02-03T09:11:20Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,94,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
94,2025-02-03T09:11:18Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,93,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
93,2025-02-03T09:11:16Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,92,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
92,2025-02-03T09:11:14Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,91,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
91,2025-02-03T09:11:13Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,90,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 716)",null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
optimize small_files

path,metrics
dbfs:/user/hive/warehouse/small_files,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1738575122214, 1738575124396, 8, 0, null, List(0, 0), 2, 2, 0, 0, null)"


In [0]:
%sql
DESC HISTORY small_files
-- optimized file has been created

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
100,2025-02-03T09:11:55Z,5755764547042441,inaya998877@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(4239566892388298),0203-052121-nggyovx3,99,SnapshotIsolation,false,"Map(numRemovedFiles -> 99, numRemovedBytes -> 70907, p25FileSize -> 1515, numDeletionVectorsRemoved -> 0, minFileSize -> 1515, numAddedFiles -> 1, maxFileSize -> 1515, p75FileSize -> 1515, p50FileSize -> 1515, numAddedBytes -> 1515)",null,Databricks-Runtime/15.4.x-scala2.12
99,2025-02-03T09:11:28Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,98,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
98,2025-02-03T09:11:26Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,97,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
97,2025-02-03T09:11:23Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,96,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
96,2025-02-03T09:11:22Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,95,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
95,2025-02-03T09:11:20Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,94,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
94,2025-02-03T09:11:18Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,93,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
93,2025-02-03T09:11:16Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,92,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
92,2025-02-03T09:11:14Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,91,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 717)",null,Databricks-Runtime/15.4.x-scala2.12
91,2025-02-03T09:11:13Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4239566892388298),0203-052121-nggyovx3,90,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 716)",null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
SELECT AVG(id) FROM small_files;
-- now query has become faster

avg(id)
50.0
